In [ ]:
import pandas as pd
import os
import glob

In [ ]:
path = r'../data/historical_data'
os.chdir(path)
hist_list = glob.glob('*.csv')
# df_fx = pd.read_csv(path+ "/" + fx_csv)

In [ ]:
pair_list = list(set([pair[:6] for pair in hist_list]))

In [ ]:
df_pair = pd.DataFrame()
start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2020-01-01')

fx_csv = list(filter(lambda x: x[:6] == pair_list[1], hist_list))
print(pair_list[1])

df_mt4 = pd.read_csv(fx_csv[0], names=['date', 'time', 'open', 'max', 'min', 'close'], index_col = False)
df_mt4['date_time'] = pd.to_datetime(df_mt4['date']+ ' '+ df_mt4['time'])
df_mt4 = df_mt4[(df_mt4.date_time >= start_date) & (df_mt4.date_time < end_date)]
df_mt4['data_type'] = 'mt4' if len(fx_csv[0]) > 20 else 'dwx'
df_mt4.drop(['date', 'time'], axis=1, inplace = True)
df_mt4.set_index('date_time', inplace = True)

df_dwx = pd.read_csv(fx_csv[1], names=['date', 'time', 'open', 'max', 'min', 'close'], index_col = False)
df_dwx['date_time'] = pd.to_datetime(df_dwx['date']+ ' '+ df_dwx['time'])
df_dwx = df_dwx[(df_dwx.date_time >= start_date) & (df_dwx.date_time < end_date)]
df_dwx['data_type'] = 'mt4' if len(fx_csv[1]) > 20 else 'dwx'
df_dwx.drop(['date', 'time'],  axis=1, inplace = True)
df_dwx.set_index('date_time', inplace = True)

In [ ]:
df_pair = df_mt4.join(df_dwx, lsuffix = '_{}'.format(df_mt4.data_type[0]), rsuffix = '_{}'.format(df_dwx.data_type[0]))

In [ ]:
df_pair.drop(['data_type_dwx', 'data_type_mt4'], axis = 1, inplace = True)

In [ ]:
df_pair.dropna(inplace = True)

In [ ]:
df_pair['open_diff'] = df_pair.open_dwx - df_pair.open_mt4
df_pair['max_diff'] = df_pair.max_dwx - df_pair.max_mt4
df_pair['min_diff'] = df_pair.min_dwx - df_pair.min_mt4
df_pair['close_diff'] = df_pair.close_dwx - df_pair.close_mt4

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=df_pair.index,
                y=df_pair.min_dwx,
                name='DWX',
                line_color='red'))

fig.add_trace(go.Scatter(
                x=df_pair.index,
                y=df_pair.min_mt4,
                name='MT4',
                line_color= 'black'))

fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(df_pair, x="close_diff")
fig.show()